- https://keras.io/applications/
- https://github.com/yang-zhang/courses/blob/scratch/deeplearning1/nbs/lesson2.ipynb

In [33]:
import math
import os
import datetime

import numpy as np
import pandas as pd

from keras.preprocessing import image
from keras.layers import Input, Lambda, Dense, Dropout, Flatten
from keras.models import Model, Sequential, load_model

from keras.utils import to_categorical
from keras.optimizers import RMSprop, SGD

from keras.applications import xception

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss, accuracy_score

from secrets import KAGGLE_USER, KAGGLE_PW

In [3]:
competition_name = 'dog-breed-identification'
data_dir = '/opt/notebooks/data/' + competition_name + '/preprocessed'

gen = image.ImageDataGenerator()
batch_size = 16
target_size=(299, 299)

def add_preprocess(base_model, preprocess_func, inputs_shape=(299, 299, 3)):
    inputs = Input(shape=inputs_shape)
    x = Lambda(preprocess_func)(inputs)
    outputs = base_model(x)
    model = Model(inputs, outputs)
    return model

### train

#### first fine-tune last layer

In [4]:
base_model = xception.Xception(weights='imagenet', include_top=False, pooling='avg')
inputs = Input(shape=(299, 299, 3))
x = Lambda(xception.preprocess_input)(inputs)
x = base_model(x)
outputs = Dense(120, activation='softmax', name='predictions')(x)
model_ft = Model(inputs, outputs)

In [5]:
for layer in base_model.layers:
    layer.trainable = False

In [6]:
batches = gen.flow_from_directory(data_dir+'/train', target_size=target_size, batch_size=batch_size)
batches_val = gen.flow_from_directory(data_dir+'/valid', shuffle=False, target_size=target_size, batch_size=batch_size)

nb_batches = math.ceil(batches.n/batch_size)
nb_batches_val = math.ceil(batches_val.n/batch_size)

y_encode = batches.classes
y_val_encode = batches_val.classes

y = to_categorical(batches.classes)
y_val = to_categorical(batches_val.classes)


Found 8222 images belonging to 120 classes.
Found 2000 images belonging to 120 classes.


In [7]:
model_ft.compile(optimizer=RMSprop(), loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
model_ft.fit_generator(batches, 
                    steps_per_epoch=nb_batches, 
                    epochs=3,
                    validation_data=batches_val,
                    validation_steps=nb_batches_val
                   )

Epoch 1/3
514/514 [==============================] - 145s - loss: 1.0520 - acc: 0.7572 - val_loss: 0.4410 - val_acc: 0.8595
Epoch 2/3
514/514 [==============================] - 144s - loss: 0.4307 - acc: 0.8658 - val_loss: 0.4363 - val_acc: 0.8670
Epoch 3/3
514/514 [==============================] - 144s - loss: 0.3752 - acc: 0.8830 - val_loss: 0.4637 - val_acc: 0.8670


#### then fine-tune more layers

In [9]:
# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

0 input_1
1 block1_conv1
2 block1_conv1_bn
3 block1_conv1_act
4 block1_conv2
5 block1_conv2_bn
6 block1_conv2_act
7 block2_sepconv1
8 block2_sepconv1_bn
9 block2_sepconv2_act
10 block2_sepconv2
11 block2_sepconv2_bn
12 conv2d_1
13 block2_pool
14 batch_normalization_1
15 add_1
16 block3_sepconv1_act
17 block3_sepconv1
18 block3_sepconv1_bn
19 block3_sepconv2_act
20 block3_sepconv2
21 block3_sepconv2_bn
22 conv2d_2
23 block3_pool
24 batch_normalization_2
25 add_2
26 block4_sepconv1_act
27 block4_sepconv1
28 block4_sepconv1_bn
29 block4_sepconv2_act
30 block4_sepconv2
31 block4_sepconv2_bn
32 conv2d_3
33 block4_pool
34 batch_normalization_3
35 add_3
36 block5_sepconv1_act
37 block5_sepconv1
38 block5_sepconv1_bn
39 block5_sepconv2_act
40 block5_sepconv2
41 block5_sepconv2_bn
42 block5_sepconv3_act
43 block5_sepconv3
44 block5_sepconv3_bn
45 add_4
46 block6_sepconv1_act
47 block6_sepconv1
48 block6_sepconv1_bn
49 block6_sepconv2_act
50 block6_sepconv2
51 block6_sepconv2_bn
52 block6_sepcon

In [11]:
# we chose to train the top 2 xception blocks, i.e. we will freeze
# the first 115 layers and unfreeze the rest:
for layer in base_model.layers[:116]:
   layer.trainable = False
for layer in base_model.layers[116:]:
   layer.trainable = True


In [13]:
# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
model_ft.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
model_ft.fit_generator(batches, 
                    steps_per_epoch=nb_batches, 
                    epochs=3,
                    validation_data=batches_val,
                    validation_steps=nb_batches_val
                   )


Epoch 1/3
514/514 [==============================] - 163s - loss: 0.3150 - val_loss: 0.4288
Epoch 2/3
514/514 [==============================] - 162s - loss: 0.2931 - val_loss: 0.4120
Epoch 3/3
514/514 [==============================] - 162s - loss: 0.2833 - val_loss: 0.4031


In [21]:
model_ft.save(data_dir+'/results/ft_3epochs_%s.h5' %  datetime.datetime.now().strftime('%Y-%m-%d-%H-%M'))  

# returns a compiled model
# identical to the previous one
# model_ft = load_model('my_model.h5')

In [22]:
model_ft.fit_generator(batches, 
                    steps_per_epoch=nb_batches, 
                    epochs=3,
                    validation_data=batches_val,
                    validation_steps=nb_batches_val
                   )


Epoch 1/3
514/514 [==============================] - 162s - loss: 0.2740 - val_loss: 0.4019
Epoch 2/3
514/514 [==============================] - 162s - loss: 0.2693 - val_loss: 0.4016
Epoch 3/3
514/514 [==============================] - 162s - loss: 0.2529 - val_loss: 0.3998


In [23]:
model_ft.save(data_dir+'/results/ft_6epochs_%s.h5' %  datetime.datetime.now().strftime('%Y-%m-%d-%H-%M'))  


In [41]:
model_ft = load_model(data_dir+'/results/ft_6epochs_2017-11-10-19-27.h5')

### predict

In [42]:
batches_test = gen.flow_from_directory(data_dir+'/test', shuffle=False, target_size=target_size, batch_size=batch_size)

Found 10357 images belonging to 1 classes.


In [43]:
nb_batches_test = math.ceil(batches_test.n/batch_size)

In [44]:
pred = model_ft.predict_generator(batches_test, steps=nb_batches_test, verbose=1)

648/648 [==============================] - 160s   


In [45]:
test_ids = [f.split('/')[1].split('.')[0] for f in batches_test.filenames]

In [46]:
subm=pd.DataFrame(np.hstack([np.array(test_ids).reshape(-1, 1), pred]))
labels = pd.read_csv(data_dir+'/labels.csv')
cols = ['id']+sorted(labels.breed.unique())
subm.columns = cols

In [47]:
description = 'xception_data_finetune_more_layers'
submission_file_name = data_dir+'/results/%s_%s.csv' % (description,
                                                        datetime.datetime.now().strftime('%Y-%m-%d-%H-%M')
                                                       )
subm.to_csv(submission_file_name, index=False)

### submit

In [48]:
!kg config -u $KAGGLE_USER -p $KAGGLE_PW -c $competition_name

In [49]:
!kg submit $submission_file_name -m $description

0.39939
